In [5]:
alert_json = {
    "rule": {
        "id": "60122",
        "description": "Suspicious logon attempt detected",
        "level": "high",
        "mitre": {
            "tactic": ["Initial Access", "Execution"],
            "technique": ["T1078", "T1059"]
        }
    },
    "agent": {
        "name": "Agent001"
    },
    "timestamp": "2023-10-01T12:34:56Z",
    "data": {
        "win": {
            "system": {
                "eventID": "4625",
                "providerName": "Microsoft-Windows-Security-Auditing"
            },
            "eventdata": {
                "logonType": "3",
                "ipAddress": "192.168.1.100",
                "failureReason": "Unknown user name or bad password"
            }
        }
    },
    "location": "Server Room 1"
}

In [14]:
import sys
import json
import requests
import configparser
from requests.auth import HTTPBasicAuth

# Read configuration from config.ini
config = configparser.ConfigParser()
config.read('config.ini')

CHAT_ID = config['telegram']['CHAT_ID']
HOOK_URL = config['telegram']['HOOK_URL']

# Extract common data fields
alert_level = alert_json['rule']['level'] if 'level' in alert_json['rule'] else "N/A"
description = alert_json['rule']['description'] if 'description' in alert_json['rule'] else "N/A"
agent = alert_json['agent']['name'] if 'name' in alert_json['agent'] else "N/A"
rule_id = alert_json['rule']['id'] if 'id' in alert_json['rule'] else "N/A"

# Function to load and fill the template from JSON
def load_and_fill_template(template, data):
    try:
        return template.format(**data)
    except KeyError as e:
        return f"Missing placeholder in template: {str(e)}"

# Function to load data mapping from JSON file
def load_data_and_template_from_json(json_path, alert_json):
    try:
        with open(json_path, 'r') as json_file:
            template_data = json.load(json_file)
        
        # Extract template
        template = template_data.get('template', '')
        
        # Dynamically evaluate the field expressions
        fields = template_data.get('fields', {})
        data = {key: eval(value) for key, value in fields.items()}
        
        return template, data
    except FileNotFoundError:
        return None, f"Data file not found: {json_path}"
    except Exception as e:
        return None, f"Error loading data from JSON: {str(e)}"

# Define a function to get message data based on rule_id
def get_message_data(rule_id, alert_json):
    if rule_id == "60122":
        # Load data and template from JSON file
        template, data = load_data_and_template_from_json('60122.json', alert_json)
        if isinstance(data, str):  # If there was an error loading the data
            return {
                'chat_id': CHAT_ID,
                'text': data
            }
        message = load_and_fill_template(template, data)
        return {
            'chat_id': CHAT_ID,
            'text': message
        }
    # Add more rule_id cases here as needed
    else:
        # Fallback in case no template is found for a specific rule_id
        return {
            'chat_id': CHAT_ID,
            'text': f"Description: {description}\nAlert Level: {alert_level}\nAgent: {agent}"
        }

# Get message data based on rule_id
msg_data = get_message_data(rule_id, alert_json)
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

# Send the request
response = requests.post(HOOK_URL, headers=headers, data=json.dumps(msg_data))
print(response)

<Response [200]>
